## Abstract

This project demonstrates best-practice patterns for containerizing a small microservice application: a React frontend served by Nginx, an Express backend and a PostgreSQL database. It includes health checks, seeding, and an example OpenShift deployment. The goal is reproducibility, clear separation of concerns and straightforward local development using `docker-compose`.

## Architecture (high level)

Frontend (Nginx/React) → Backend (Node/Express) → Database (Postgres).

- Frontend: static assets served by Nginx; proxies `/api/*` to backend.
- Backend: REST API, DB access via `pg`, converts DB types for JSON.
- Database: Postgres with initialization script that seeds sample products (includes `image_url`).

## Implementation notes and design decisions

- Use container hostnames for internal communication (Nginx proxy + service DNS).
- Normalize database types (e.g., DECIMAL→Number) in the backend to avoid frontend runtime errors.
- Keep frontend API calls relative (`/api/...`) so that Nginx can proxy in containerized environments and `REACT_APP_API_URL` can be used for local development.
- Seed the DB via `database/init.sql` and make schema changes in that script for deterministic local setups.

## Important backend snippet — normalize `price` before JSON response

Explanation: PostgreSQL numeric/decimal types are returned as strings by the `pg` client by default. Convert `price` to a numeric JavaScript value with `parseFloat` to allow numeric operations and formatting in the frontend (e.g., `toFixed`).

In [ ]:
// backend: server.js — normalization helper
const normalizeProduct = (product) => ({
  ...product,
  price: parseFloat(product.price)
});

// Example usage in route:
app.get('/api/products', async (req, res) => {
  const result = await client.query('SELECT * FROM products ORDER BY id');
  res.json(result.rows.map(normalizeProduct));
});

## Important frontend snippet — image handling and fallback

Explanation: Display product images when available. Hide the fallback emoji once the image successfully loads (`onLoad`). If an image fails to load, the `onError` handler hides the broken image and the fallback remains visible. This prevents emoji overlay when images are present.

In [ ]:
// frontend: src/components/ProductCard.js (core part)
function ProductCard({ product }) {
  const handleImageLoad = (e) => {
    const icon = e.target.parentElement.querySelector('.product-icon');
    if (icon) icon.style.display = 'none';
  };

  const handleImageError = (e) => {
    e.target.style.display = 'none';
  };

  return (
    <div className="product-card">
      <div className="product-image">
        {product.image_url ? (
          <img src={product.image_url} alt={product.name} onLoad={handleImageLoad} onError={handleImageError} />
        ) : null}
        <span className="product-icon">📦</span>
      </div>
      ...
    </div>
  );
}

## Web server proxy config (Nginx) — relevant excerpt

Explanation: Nginx serves static files and proxies `/api` requests to the backend service `backend:5000` inside Docker Compose. This allows the frontend to use relative API paths in production containers.

In [ ]:
server {
  listen 80;
  location / {
    root /usr/share/nginx/html;
    try_files $uri $uri/ /index.html;
  }

  location /api {
    proxy_pass http://backend:5000;
    proxy_set_header Host $host;
    proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
  }
}

## Database schema snippet (seed)

Explanation: The `init.sql` script creates the `products` table and inserts sample rows, including `image_url`. The DB is re-seeded when the named volume is removed and containers restarted.

In [ ]:
CREATE TABLE IF NOT EXISTS products (
  id SERIAL PRIMARY KEY,
  name VARCHAR(255) NOT NULL,
  description TEXT,
  price DECIMAL(10,2) NOT NULL,
  image_url VARCHAR(512),
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

INSERT INTO products (name, description, price, image_url) VALUES ('Laptop','High-performance laptop',999.99,'https://example.com/laptop.jpg');

## Orchestration excerpt (`docker-compose`)

Explanation: Compose file defines three services on a user-defined bridge network and a named volume for DB persistence. Health checks and `depends_on` ensure ordered startup.

In [ ]:
services:
  database:
    image: postgres:15-alpine
    volumes:
      - db_data:/var/lib/postgresql/data
  backend:
    build: ./backend
    environment:
      - DB_HOST=database
  frontend:
    build: ./frontend
    ports:
      - "80:80"
volumes:
  db_data:

## Testing & verification commands

Run these from the project root on Windows (PowerShell):

```powershell
docker-compose up -d --build
Invoke-WebRequest -Uri "http://localhost:5000/api/products" -UseBasicParsing
docker exec -it product-database psql -U postgres -d productdb -c "SELECT name, image_url FROM products;"
docker-compose logs -f
```

## Deployment notes & reproducibility

- To reproduce the seeded DB state, remove the named volume and restart: `docker-compose down -v && docker-compose up -d --build`.
- For production, replace image URLs with a CDN or local storage, secure secrets and enable TLS.

## Conclusions & next steps

This notebook provides a structured report and the key code excerpts needed for an academic submission. Next steps you may request:
- Expand appendix with full files (server.js, init.sql, nginx.conf)
- Generate a PDF report from this notebook
- Add unit tests for the backend API endpoints